Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [26]:
from random import choices, choice, random, randint
from copy import copy

import lab9_lib

In [35]:
GENOME_LENGTH = 10

fitness = lab9_lib.make_problem(GENOME_LENGTH)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10110100101011011010110001110001110100111111101111: 39.34%
00111111101010000011100100010111110011010010100011: 15.33%
11011100100011010001000011100110110010100110010100: 7.34%
11011111000100110011110111011110111101011011110001: 19.11%
01000110110100110011011111111111011001010001100100: 23.33%
01001001110111010011011100010001011110001110101110: 9.33%
00101010011111011001000010011110110100001111110011: 15.33%
11010110010101110111110011110100010000011110101101: 9.13%
00100010100110000111100111100111111010011001110100: 35.56%
01011011011011111000000111001101100110001000111111: 23.34%
10


In [20]:
OFFSPRING_SIZE = 20
MUTATION_PROBABILITY = .15
POPULATION_SIZE = 30
TOURNAMENT_SIZE = 2

In [15]:
population = [choices([0, 1], k=50)] * POPULATION_SIZE

In [27]:
def tournament_selection(population):
    return max(
        [choice(population) for _ in range(TOURNAMENT_SIZE)], key=lambda i: fitness(i)
    )


def one_cut_xover(ind1, ind2):
    cut_point = randint(0, GENOME_LENGTH - 1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    return offspring

def mutate(ind):
    offspring = copy(ind)
    pos = randint(0, GENOME_LENGTH-1)
    offspring[pos] = not offspring[pos]
    return offspring

In [37]:
for generation in range(300):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = tournament_selection(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = tournament_selection(population)
            p2 = tournament_selection(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population.sort(key=lambda i: fitness(i), reverse=True)
    population = population[:POPULATION_SIZE]
    print(fitness(population[0]))

    print(fitness.calls)

0.41778
37617
0.41778
37740
0.41778
37865
0.41778
37992
0.41778
38121
0.41778
38248
0.41778
38375
0.41778
38494
0.41778
38617
0.41778
38744
0.41778
38873
0.41778
38998
0.41778
39129
0.41778
39254
0.41778
39377
0.41778
39502
0.41778
39627
0.41778
39748
0.41778
39877
0.41778
40002
0.41778
40133
0.41778
40262
0.41778
40387
0.41778
40506
0.41778
40633
0.41778
40754
0.41778
40877
0.41778
41002
0.41778
41123
0.41778
41252
0.41778
41377
0.41778
41504
0.41778
41629
0.41778
41758
0.41778
41881
0.41778
42008
0.41778
42131
0.41778
42258
0.41778
42381
0.41778
42502
0.41778
42623
0.41778
42748
0.41778
42871
0.41778
42994
0.41778
43119
0.41778
43242
0.41778
43369
0.41778
43496
0.41778
43621
0.41778
43750
0.41778
43873
0.41778
43998
0.41778
44117
0.41778
44234
0.41778
44357
0.41778
44484
0.41778
44607
0.41778
44730
0.41778
44857
0.41778
44982
0.41778
45107
0.41778
45238
0.41778
45365
0.41778
45496
0.41778
45619
0.41778
45746
0.41778
45871
0.41778
45994
0.41778
46111
0.41778
46238
0.41778
46363
0.4177